In [120]:
import pandas as pd
import numpy as np
import random 

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 200)
num_ofDesiredRecords = 1000

In [12]:
def createRandomPhoneNumber_SAA():
    range_start = 10**9
    range_end = 10**10 -1
    num = str(random.randint(range_start,range_end))
    num_str = "{0} {1}-{2}".format(num[:3], num[3:7],num[7:])
    return num_str

In [92]:
def createRandomEmail(record,domainList=['@gmail.com','@stanfordalumni.org','@alumni.stanford.edu']):
    domain = random.choice(domainList)
    fName = record['first_name'].strip().lower()
    LName = record['last_name'].strip().lower()
    formats = []
    
    #Example: John Doe 123
    formats.append(fName[0]+LName+str(random.randint(10,9999))) #jdoe123
    formats.append(fName+LName+str(random.randint(10,9999))) #johndoe123
    formats.append(fName+LName[0]+str(random.randint(10,9999))) #johnd123
    formats.append(fName[0]+'.'+LName+str(random.randint(10,9999))) #j.doe123
    formats.append(fName+'.'+LName+str(random.randint(10,9999))) #john.doe123
    
    return random.choice(formats)+domain

In [89]:
createRandomEmail('John','Doe')

'j.doe9294@gmail.com'

In [42]:
common_male_Fnames = [
    'Liam',
    'Noah',
    'Oliver',
    'Elijah',
    'James',
    'William',
    'Benjamin',
    'Lucas',
    'Henry',
    'Theodore',
]

In [43]:
common_female_Fnames = [
    'Mary',
    'Patricia',
    'Jennifer',
    'Linda',
    'Elizabeth',
    'Barbara',
    'Susan',
    'Jessica',
    'Sarah',
    'Karen',
]

In [44]:
common_FNames = []
common_FNames.extend(common_female_Fnames)
common_FNames.extend(common_male_Fnames)

In [45]:
common_LNames = ['Smith',
'Johnson',
'Williams',
'Brown',
'Jones',
'Garcia',
'Miller',
'Davis',
'Rodriguez',
'Martinez',]

In [46]:
df_SAA = pd.read_excel("./data/SAA Pride member reports headings.xlsx").append([None]*num_ofDesiredRecords)

C:\Users\saads\AppData\Local\Temp\ipykernel_17944\884349795.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SAA = pd.read_excel("./data/SAA Pride member reports headings.xlsx").append([None]*num_ofDesiredRecords)


In [126]:
df_SAA

,pref_mail_name,pref_class_year,home_city,home_state_code,home_country,home_phone_area_code,home_phone_number,home_email_address,bus_city,bus_state_code,bus_country,bus_phone_area_code,bus_phone_number,bus_email_address,first_name,last_name,pref_name_sort,email_switch,saa_email_address,gsb_email_address,other_email_address,pref_phone_area_code,pref_phone_number,pref_phone_addr_type,memb_status_desc,short_degree_string,parent_degree_string,short_degree_string_spouse,parent_degree_string_spouse,primary_sort_name,plan_name,primary_ind,0
0,NaN,NaN,NaN,NaN,NaN,NaN,*,olivermiller3453@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,Oliver,Miller,NaN,NaN,*,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,NaN,NaN,NaN,NaN,NaN,NaN,None,lucasr7304@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,Lucas,Rodriguez,NaN,NaN,lucasr1462@alumni.stanford.edu,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,NaN,NaN,NaN,NaN,NaN,NaN,186 1448-765,smartinez6171@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,Susan,Martinez,NaN,NaN,s.martinez3252@alumni.stanford.edu,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,NaN,NaN,NaN,NaN,NaN,NaN,*,w.smith9748@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,William,Smith,NaN,NaN,*,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,NaN,NaN,NaN,NaN,NaN,NaN,685 8096-158,lindagarcia264@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,Linda,Garcia,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
5,NaN,NaN,NaN,NaN,NaN,NaN,887 7671-703,o.williams2750@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,Oliver,Williams,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
6,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,Jessica,Rodriguez,NaN,NaN,*,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
7,NaN,NaN,NaN,NaN,NaN,NaN,*,None,NaN,NaN,NaN,NaN,NaN,NaN,Noah,Brown,NaN,NaN,*,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
8,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,Noah,Garcia,NaN,NaN,ngarcia3519@stanfordalumni.org,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
9,NaN,NaN,NaN,NaN,NaN,NaN,None,lucasjones7954@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,Lucas,Jones,NaN,NaN,None,lucasjones5026@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [47]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(common_FNames))
    
df_SAA.first_name = __

In [48]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(common_LNames))
    
df_SAA.last_name = __

In [122]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choices([createRandomPhoneNumber_SAA(),'*',None],weights=(1,5,5))[0])
    
df_SAA.home_phone_number = __

In [123]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(['*',None,createRandomEmail(df_SAA.iloc[_])]))
    
df_SAA.home_email_address = __

In [124]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(['*',None,createRandomEmail(df_SAA.iloc[_],['@stanfordalumni.org','@alumni.stanford.edu'])]))
    
df_SAA.saa_email_address = __

In [125]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choices([createRandomEmail(df_SAA.iloc[_]),None],weights=(1, 50))[0])
    
df_SAA.gsb_email_address = __